In [ ]:
import cv2
import numpy as np
import sys
from ultralytics import YOLO
from sklearn.cluster import KMeans

In [ ]:
def get_grass_color(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_green = np.array([30, 40, 40])
    upper_green = np.array([80, 255, 255])

    mask = cv2.inRange(hsv, lower_green, upper_green)

    masked_img = cv2.bitwise_and(img, img, mask=mask)
    grass_color = cv2.mean(img, mask=mask)
    return grass_color[:3]

In [ ]:
def get_players_boxes(result):
    players_imgs = []
    players_boxes = []
    goal = [] 
    ball = []
    for box in result.boxes:
        label = int(box.cls.numpy()[0])
        if label == 0:
            x1, y1, x2, y2 = map(int, box.xyxy[0].numpy())
            player_img = result.orig_img[y1: y2, x1: x2]
            players_imgs.append(player_img)
            players_boxes.append(box)
        if label == 1:
            goal.append(box)
        if label == 2:
            ball.append(box)
    return players_imgs, players_boxes ,goal ,ball

In [ ]:
def get_kits_colors(players, grass_hsv=None, frame=None):
    kits_colors = []
    if grass_hsv is None:
        grass_color = get_grass_color(frame)
        grass_hsv = cv2.cvtColor(np.uint8([[list(grass_color)]]), cv2.COLOR_BGR2HSV)
    for player_img in players:
        hsv = cv2.cvtColor(player_img, cv2.COLOR_BGR2HSV)

        lower_green = np.array([grass_hsv[0, 0, 0] - 10, 40, 40])
        upper_green = np.array([grass_hsv[0, 0, 0] + 10, 255, 255])

        mask = cv2.inRange(hsv, lower_green, upper_green)

        mask = cv2.bitwise_not(mask)
        upper_mask = np.zeros(player_img.shape[:2], np.uint8)
        upper_mask[0:player_img.shape[0]//2, 0:player_img.shape[1]] = 255
        mask = cv2.bitwise_and(mask, upper_mask)

        kit_color = np.array(cv2.mean(player_img, mask=mask)[:3])

        kits_colors.append(kit_color)
    return kits_colors

In [ ]:
def get_kits_classifier(kits_colors):
    kits_kmeans = KMeans(n_clusters=2, n_init=10)
    kits_kmeans.fit(kits_colors);
    return kits_kmeans

In [ ]:
def classify_kits(kits_classifer, kits_colors):
    team = kits_classifer.predict(kits_colors)
    return team

In [ ]:
def calculate_distance(box1, box2):
    center1 = np.array([(box1['x'] + box1['w']) / 2, (box1['y'] + box1['h']) / 2])
    center2 = np.array([(box2['x'] + box2['w']) / 2, (box2['y'] + box2['h']) / 2])
    distance = np.linalg.norm(center1 - center2)
    return distance

In [ ]:
def get_nearset_players(players_boxes, goal, kits_clf, img, grass_hsv):
    
    x1_gk, y1_gk, x2_gk, y2_gk = map(int, goal[0].xyxy[0].numpy())
    goal_box = {'x': x1_gk, 'y': y1_gk, 'w': x2_gk , 'h': y2_gk }
    players_distances = []
    
    for player in players_boxes:
        x1, y1, x2, y2 = map(int, player.xyxy[0].numpy())
        player_box = {'x': x1, 'y': y1, 'w': x2 , 'h': y2 }
        
        distance_to_goal = calculate_distance(goal_box, player_box)
        
        players_distances.append({
            'box': player.xyxy[0].numpy(),
            'distance': distance_to_goal,
        })

    players_distances.sort(key=lambda player: player['distance'])

    for player in players_distances:
        kit_color = get_kits_colors([img[int(player['box'][1]): int(player['box'][3]), int(player['box'][0]): int(player['box'][2])]], grass_hsv)
        player['team'] = classify_kits(kits_clf, kit_color)

    team_1_players = [player for player in players_distances if player['team'] == 0]
    team_2_players = [player for player in players_distances if player['team'] == 1]
    
    nearest_player_team_1 = team_1_players[0] if team_1_players else None
    nearest_player_team_2 = team_2_players[0] if team_2_players else None
    return nearest_player_team_1, nearest_player_team_2

In [ ]:
def annotate_image(img , model, color):
    kits_clf = None
    grass_hsv = None
    
    result = model(img, verbose=False)[0]
    img = result.orig_img.copy()
    
    players_imgs, players_boxes , goal , ball = get_players_boxes(result)
    
    kits_colors = get_kits_colors(players_imgs, grass_hsv, img)
    kits_clf = get_kits_classifier(kits_colors)
    
    grass_color = get_grass_color(result.orig_img)
    grass_hsv = cv2.cvtColor(np.uint8([[list(grass_color)]]), cv2.COLOR_BGR2HSV)
    
    team_1, team_2 = get_nearset_players(players_boxes, goal, kits_clf , img, grass_hsv)
    x1, y1, x2, y2 = map(int, ball[0].xyxy[0].numpy())
    
    cv2.rectangle(img, (int(team_1['box'][0]), int(team_1['box'][1])), (int(team_1['box'][2]), int(team_1['box'][3])), color, 2)
    cv2.rectangle(img, (int(team_2['box'][0]), int(team_2['box'][1])), (int(team_2['box'][2]), int(team_2['box'][3])), color, 2)
    cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
    
    cv2.imshow('Match',img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
if __name__ == "__main__":
    model = YOLO(r"E:\vision_task\vision_task\best.pt")
#     img =  r'E:\vision_task\vision_task\test1.PNG'
#     img =  r'E:\vision_task\vision_task\test3.jpg'
#     img =  r'E:\vision_task\vision_task\test2.PNG'
    
    color = (255, 0, 0) 
    annotate_image(img, model , color)